# Step 2B: Model Building

This notebook constructs a machine learning model designed to predict component failure in the machine. You must run the `2_feature_engineering` notebook before running this notebook.

You can either `Run All` cells, or use the Databricks CLI to create a Databricks Job to do the same process automatically.

To test the model constructed in this notebook, the `2b_model_testing` notebook has been included in the repostiory and copied to your Azure Databricks Workspace. You must run this notebook before running the model testing notebook, which calculates some model performance metrics for this predictive maintenance model.

Using the labeled feature data set constructed in the `2a_feature_engineering` Jupyter notebook, this notebook loads the data from the training data set. Then builds a machine learning model (either a decision tree classifier or a random forest classifier) to predict when different components within our machine population will fail. We store the model for deployment on the Databricks DBFS file system (`dbfs:/storage/models/` + model_type + `.pqt`) in parquet format for use in testing (`2b_model_testing`) and scoring (`3b_model_scoring`) operations.

**Note:** This notebook will take about 2-4 minutes to execute all cells, depending on the compute configuration you have setup.

In [2]:
# import the libraries
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
# for creating pipelines and model
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# This is the final feature data file.
training_table= 'training_data'
model_type = 'RandomForest' # Use 'DecisionTree' or 'GBTClassifier' or 'RandomForest'

In [3]:
dbutils.widgets.removeAll()
dbutils.widgets.text("training_table",training_table)
dbutils.widgets.text("model", model_type)

Load the training data.

In [5]:
train_data = spark.table(dbutils.widgets.get("training_table"))

# Prepare the Training data

A fundamental practice in machine learning is to calibrate and test your model parameters on data that has not been used to train the model. Evaluation of the model requires splitting the available data into a training portion, a calibration portion and an evaluation portion. Typically, 80% of data is used to train the model and 10% each to calibrate any parameter selection and evaluate your model.

In general random splitting can be used, but since time series data have an inherent correlation between observations. For predictive maintenance problems, a time-dependent spliting strategy is often a better approach to estimate performance. For a time-dependent split, a single point in time is chosen, the model is trained on examples up to that point in time, and validated on the examples after that point. This simulates training on current data and score data collected in the future data after the splitting point is not known. However, care must be taken on labels near the split point. In this case, feature records within 7 days of the split point can not be labeled as a failure, since that is unobserved data.

In [7]:
# define list of input columns for downstream modeling

# We'll use the known label, and key variables.
label_var = ['label_e']
key_cols =['machineID','dt_truncated']

# Then get the remaing feature names from the data
input_features = train_data.columns

# We'll use the known label, key variables and 
# a few extra columns we won't need.
remove_names = label_var + key_cols + ['failure','model_encoded','model' ]

# Remove the extra names if that are in the input_features list
input_features = [x for x in input_features if x not in set(remove_names)]

Spark models require a vectorized data frame. We transform the dataset here and then split the data into a training and test set. We use this split data to train the model on 9 months of data (training data), and evaluate on the remaining 3 months (test data) going forward.

In [9]:
# assemble features
va = VectorAssembler(inputCols=(input_features), outputCol='features')
train_data = va.transform(train_data).select('machineID','dt_truncated','label_e','features')

# set maxCategories so features with > 10 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", 
                               outputCol="indexedFeatures", 
                               maxCategories=10).fit(train_data)

# fit on whole dataset to include all labels in index
labelIndexer = StringIndexer(inputCol="label_e", outputCol="indexedLabel").fit(train_data)

training = train_data

# Classification models

A particular problem in predictive maintenance is machine failures are usually rare occurrences compared to normal operation. This is fortunate for the business as maintenance and saftey issues are few, but causes an imbalance in the label distribution. This imbalance leads to poor performance as algorithms tend to classify majority class examples at the expense of minority class, since the total misclassification error is much improved when majority class is labeled correctly. This causes low recall or precision rates, although accuracy can be high. It becomes a larger problem when the cost of false alarms is very high. To help with this problem, sampling techniques such as oversampling of the minority examples can be used. These methods are not covered in this notebook. Because of this, it is also important to look at evaluation metrics other than accuracy alone.

We will build and compare two different classification model approaches:

 - **Decision Tree Classifier**: Decision trees and their ensembles are popular methods for the machine learning tasks of classification and regression. Decision trees are widely used since they are easy to interpret, handle categorical features, extend to the multiclass classification setting, do not require feature scaling, and are able to capture non-linearities and feature interactions.

 - **Random Forest Classifier**: A random forest is an ensemble of decision trees. Random forests combine many decision trees in order to reduce the risk of overfitting. Tree ensemble algorithms such as random forests and boosting are among the top performers for classification and regression tasks.

The next code block creates the model. You can choose between a _DecisionTree_ or _RandomForest_ by setting the 'model_type' variable. We have also included a series of model hyperparameters to guide your exploration of the model space.

In [11]:
model_type = dbutils.widgets.get("model")

# train a model.
if model_type == 'DecisionTree':
  model = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures",
                                 # Maximum depth of the tree. (>= 0) 
                                 # E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'
                                 maxDepth=15,
                                 # Max number of bins for discretizing continuous features. 
                                 # Must be >=2 and >= number of categories for any categorical feature.
                                 maxBins=32, 
                                 # Minimum number of instances each child must have after split. 
                                 # If a split causes the left or right child to have fewer than 
                                 # minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.
                                 minInstancesPerNode=1, 
                                 # Minimum information gain for a split to be considered at a tree node.
                                 minInfoGain=0.0, 
                                 # Criterion used for information gain calculation (case-insensitive). 
                                 # Supported options: entropy, gini')
                                 impurity="gini")

  ##=======================================================================================================================
  ## GBTClassifer is only valid for Binary Classifiers, this is a multiclass (failures 1-4) so no GBTClassifier
#elif model_type == 'GBTClassifier':
#  model = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures",
#                        maxIter=200, stepSize=0.1,
#                        maxDepth=15,
#                        maxBins=32, 
#                        minInstancesPerNode=1, 
#                        minInfoGain=0.0)
  ##=======================================================================================================================
else:
  model = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", 
                                      # Passed to DecisionTreeClassifier
                                      maxDepth=15, 
                                      maxBins=32, 
                                      minInstancesPerNode=1, 
                                      minInfoGain=0.0,
                                      impurity="gini",
                                      # Number of trees to train (>= 1)
                                      numTrees=200, 
                                      # The number of features to consider for splits at each tree node. 
                                      # Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].
                                      featureSubsetStrategy="sqrt", 
                                      # Fraction of the training data used for learning each  
                                      # decision tree, in range (0, 1].' 
                                      subsamplingRate = 0.632)

# chain indexers and model in a Pipeline
pipeline_cls_mthd = Pipeline(stages=[labelIndexer, featureIndexer, model])

# train model.  This also runs the indexers.
model_pipeline = pipeline_cls_mthd.fit(training)

## Persist the model

Here we save the model in a parquet file on DBFS.

In [13]:
# save model
model_pipeline.write().overwrite().save("dbfs:/storage/models/" + model_type + ".pqt")

# Conclusion


We have now stored the model on the Azure Databricks files system. The `2b_model_testing` notebook tests the model on new data and calculates a set of model evaluation metrics to help us know how well the model may performa in a production setting. 

The next step is to build the batch scoreing operations. The `3_Scoring_Pipeline` notebook takes parameters to define the data to be scored, and using the model created here, calulates the probability of component failure in the machine population specified.